In [1]:
import sys
from itertools import count
import random
from random import shuffle
from typing import Tuple

import pandas as pd
import numpy as np
import scipy
from scipy.sparse import coo_array
from scipy.sparse import csc_matrix
from scipy.sparse import linalg as sLA
import scipy.linalg as LA
from sklearn_extra.cluster import KMedoids
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

_ = np.seterr(invalid='ignore')

# Skupovi podataka sa kojima radimo su
- deo netflixprize dataseta (https://www.kaggle.com/datasets/rishitjavia/netflix-movie-rating-dataset?resource=download)
- lastfm dataset            (https://www.upf.edu/web/mtg/lastfm360k)
- movie lens dataset        (https://grouplens.org/datasets/movielens/)

Zbog ogranicenih racunarskih resursa posmatrali smo podskupove originalnih skupova podataka

In [2]:
def load_dataset(dataset: str, new_product=False):
    if dataset == 'movie_lens':
        ratings = pd.read_csv('data/movie_lense/ratings.csv')
        ratings = ratings.drop(columns=['timestamp'])
        if new_product:
            ratings.columns = ['productId', 'userId', 'rating'] 
        else:
            ratings.columns = ['userId', 'productId', 'rating']
    elif dataset == 'netflix':
        ratings = pd.read_csv('data/netflix/Netflix_Dataset_Rating.csv')
        if new_product:
            ratings.columns = ['productId', 'rating', 'userId']
        else:
            ratings.columns = ['userId', 'rating', 'productId']
    elif dataset == 'lastfm':
        ratings = pd.read_csv('data/lastfm/usersha1-artmbid-artname-plays.tsv', sep='\t', header=None)
        ratings.columns = ['user_hash', 'song_hash', 'artist_name', 'num_plays']
        ratings = ratings.drop(columns=['artist_name'])
        if new_product:
            ratings.columns = ['productId', 'userId', 'rating']
        else:
            ratings.columns = ['userId', 'productId', 'rating']
    else:
        raise ValueError(f'dataset {dataset} is not supported')
    
    return ratings



In [3]:
def trim_dataset(ratings: pd.DataFrame, dataset: str, max_products=100_000, max_users=1400) -> pd.DataFrame:
    if dataset == 'movie_lens':
        most_ratings = ratings['userId'].value_counts().index
        ratings = ratings[ratings.userId.isin(most_ratings[:max_products])]
        most_rated = ratings['productId'].value_counts().index
        ratings = ratings[ratings.productId.isin(most_rated[:max_users])]
    elif dataset == 'netflix':
        pass
    elif dataset == 'lastfm':
        most_ratings = ratings['userId'].value_counts().index
        ratings = ratings[ratings.userId.isin(most_ratings[:max_products])]
        most_rated = ratings['productId'].value_counts().index
        ratings = ratings[ratings.productId.isin(most_rated[:max_users])]

    return ratings

In [4]:
def clean_ids(ratings: pd.DataFrame) -> pd.DataFrame:
    usersIds = ratings.userId.unique()
    productIds = ratings.productId.unique()

    n = len(usersIds)
    m = len(productIds)

    usersIds_df = pd.DataFrame({'userId': usersIds, 'newUserId': range(n)})
    productIds_df = pd.DataFrame({'productId': productIds, 'newProductId': range(m)})

    ratings = ratings[ratings.userId.isin(usersIds)]
    ratings = ratings[ratings.productId.isin(productIds)]

    ratings = pd.merge(ratings, usersIds_df, on='userId').drop(columns=['userId'])
    ratings = pd.merge(ratings, productIds_df, on='productId').drop(columns=['productId'])

    ratings.columns = ['rating','userId', 'productId']
    return ratings


In [5]:
# Ucitavanje podataka
dataset = 'movie_lens'    # movie_lens, netflix or lastfm

ratings = load_dataset(dataset)  # add new_product=True to rate new products
ratings = trim_dataset(ratings, dataset) # netflix will not be trimmed, if we want to rate new product change max_product=1400, max_users=100_000
ratings = clean_ids(ratings)
ratings = ratings.astype({'rating': float})

num_representatives = 20
print(f'number of ratings: {ratings.shape[0]}')


number of ratings: 15468742


In [6]:
ratings.head()

,rating,userId,productId
0,5.0,0,0
1,5.0,2,0
2,4.0,3,0
3,4.0,4,0
4,5.0,5,0


In [7]:
nb_users = len(ratings.userId.unique())
print('nb users', nb_users)
nb_products = len(ratings.productId.unique())
print('nb products', nb_products)
data = coo_array((ratings['rating'], (ratings['userId'], ratings['productId'])), shape=(nb_users, nb_products)).toarray()

print('data shape ', data.shape)

nb users 100000
nb products 1400
data shape  (100000, 1400)


In [8]:
def get_C_X(representatives, data, ratings):
    # for movies in representative, make C with rating, userId, productId for that movies
    k = len(representatives)
    nb_users = ratings['userId'].max()+1

    representatives = list(representatives)
    C = data[:, representatives]
    
    # C = ratings[ratings.productId.isin(representatives)]
    # C.productId = C.productId.replace(dict(zip(representatives, count())))

    # C = coo_array((C['rating'], (C['userId'], C['productId'])), shape=(nb_users, k))

    X = LA.inv((C.T @ C + 0.1*np.eye(C.shape[1])))@C.T@data

    return C, X

# RBMF (Representativ Based Matrix Factorization)

In [9]:
def get_init_P(A: np.ndarray) -> np.ndarray:
    """function returns permutation obtained by LU decomposition of matrix A

    Args:
        A (np.ndarray): matrix on which we do LU decompositon

    Returns:
        np.ndarray: arry of row indices in permuted matrix
    """
    assert len(A.shape) == 2, "LU decomposition is only valid for 2 dimensional arrays"
    n, m = A.shape
    U = np.copy(A)
    
    P = np.arange(0, n)
    # L = np.zeros_like(P)
    for i in range(min(n, m)):
        # get the element with max absolut value
        column = U[i:, i]
        max_id = i + np.abs(U[i:, i]).argmax()
        if np.isclose(U[max_id, i], 0):
            continue
        
        # perform necessary permutations to make max el on diagonal
        P[max_id], P[i] = P[i], P[max_id]
        U[[max_id, i]] = U[[i, max_id]]

        # perform gaussian transformation
        for j in range(i+1, n):
            U[j] = (-U[j, i]/U[i, i])*U[i] + U[j]

    return P


In [10]:
def get_rbmf_representatives(k, data):
    U, s, Vh = sLA.svds(data, k)

    #print('U', U.shape)
    #print('Vh', Vh.shape)
    
    V = Vh.T
    p = get_init_P(V)

    done = False
    while not done:
        B = V @ LA.inv(V[p[:k], :])
        flat_index = np.abs(B).argmax()
        i = flat_index // B.shape[0]
        j = flat_index % B.shape[1]
        if np.abs(B[i, j] > 1):
            p[i], p[j] = p[j], p[i]
        else:
            done = True

    return p[:k]

# &rarr; Random Strategy

In [11]:
def get_random_representatives(k, ratings):
    productIds = ratings.productId.unique()
    random.shuffle(productIds)
    representatives_random = productIds[:k]
    
    return representatives_random

# &rarr; Most Ratings Strategy

In [12]:
def get_most_rated_representatives(k, ratings):
    most_ratings = ratings['productId'].value_counts().index

    representatives_most_rating = most_ratings[:k]

    return representatives_most_rating

# K-Medoids Strategy

In [13]:
def get_cluster_representatives(k):
    clusters = KMedoids(k, init='k-medoids++')
    clusters.fit(data.T)

    return clusters.medoid_indices_

# Ocena dobijenih reprezentativnih uzoraka

## coverege

 Coverage: the proportion of users (or items) which
have rated the set of representative items (or have been
rated by the representative users). This metric allows
us to measure how widely are the user or item population being covered by the representative set as a
whole

In [14]:
# videti koliko korisnika ima neku ocenu (ili koliko je proizvoda ocenio bar neki 
# reprezentativan korisnik)

# govori nam kolika je verovatnoca da nemam korisnih informacija u representative
# skupu, odnosno koliko cesto moze da se desi da korisnik nije pogledao nista od
# navedenih filmova


def coverege(ratings, representatives):
    
    k = len(representatives)
    users = pd.DataFrame({'userId': ratings['userId'].unique()})
    count = 0

    representative_ratings = ratings[ratings.productId.isin(representatives)]
    num_representative_ratings = representative_ratings.userId.value_counts()

    num_representative_ratings = pd.DataFrame(
        {
            'userId': num_representative_ratings.index,
            'numRatings': num_representative_ratings.values
        })

    user_data = pd.merge(users, num_representative_ratings, how='left', on=['userId']).fillna(0)
    
    
    return (user_data.numRatings > 0).mean()

# diversity 

Diversity:the proportion of users (or items) which
have rated less than 10% of the representative items
(or have been rated by less than 10% of the represen-
tative users). This metric reflects how much unique
information are carried by each representative

In [15]:
# za svakog korisnika naci broj ocenjenih reprezentaativnih itema
# videti koliko ih je ocenilo manje od 10%

def diversity(ratings, representatives):
    k = len(representatives)
    users = pd.DataFrame({'userId': ratings['userId'].unique()})
    count = 0

    representative_ratings = ratings[ratings.productId.isin(representatives)]
    num_representative_ratings = representative_ratings.userId.value_counts()

    num_representative_ratings = pd.DataFrame(
        {
            'userId': num_representative_ratings.index,
            'numRatings': num_representative_ratings.values
        })

    user_data = pd.merge(users, num_representative_ratings, how='left', on=['userId']).fillna(0)
    
    
    return ((user_data.numRatings < 0.1*k)).mean()

# na primer ukoliko je svaki korisnik ocenio samo mali broj itema
# ili su neki itemi beskorisni (npr niko ne ocenjuje neki film jer ga niko ne gleda)
# ili smo pronasli male grupice filmova koji karakterisu korisnike

# cinjenica da neko pogledao good will hunting mnogo manje verovatno govori o osobi od 
# cinjenice da je pogledala podmornica potemkin (crno beli nemi film)

In [16]:
k = num_representatives
representatives_random = get_random_representatives(k=k, ratings=ratings)
representatives_most_rating = get_most_rated_representatives(k=k,ratings=ratings)
representatives_cluster = get_cluster_representatives(k=k)
representatives_rbmf = get_rbmf_representatives(k=k,data=data)

In [17]:
print('coverege random', coverege(ratings, representatives_random))
print('coverege most rating', coverege(ratings, representatives_most_rating))
print('coverege cluster', coverege(ratings, representatives_cluster))
print('coverege rbmf', coverege(ratings, representatives_rbmf))

coverege random 0.73096
coverege most rating 0.98691
coverege cluster 0.91346
coverege rbmf 0.98031


In [18]:
print('diversity random', diversity(ratings, representatives_random))
print('diversity most rating', diversity(ratings, representatives_most_rating))
print('diversity cluster', diversity(ratings, representatives_cluster))
print('diversity rbmf', diversity(ratings, representatives_rbmf))

diversity random 0.58379
diversity most rating 0.03834
diversity cluster 0.2633
diversity rbmf 0.07383


# Koriscenje dobijenih modela za preporuke

### priprema podataka

Koristimo skup podataka podeljen na trening i validaciju u odnosu tom i tom.
Pratimo tri metrike


In [19]:
k = num_representatives
ratings.head()

,rating,userId,productId
0,5.0,0,0
1,5.0,2,0
2,4.0,3,0
3,4.0,4,0
4,5.0,5,0


In [20]:
# podela skupa na trening i test skup
usersIds = ratings.userId.unique()
random.shuffle(usersIds)

n = len(usersIds)
train_size = round(n*0.8)
test_size = round(n*0.2)

usersIds_train = usersIds[:train_size]
usersIds_train_df = pd.DataFrame({'userId': usersIds_train, 'newId': range(train_size)})

usersIds_test = usersIds[train_size:]
usersIds_test_df = pd.DataFrame({'userId': usersIds_test, 'newId': range(test_size)})

ratings_train = ratings[ratings.userId.isin(usersIds_train)]
ratings_train = pd.merge(ratings_train, usersIds_train_df, on='userId').drop(columns=['userId'])
ratings_train.columns = ['rating', 'productId', 'userId']

ratings_test = ratings[ratings.userId.isin(usersIds_test)]
ratings_test = pd.merge(ratings_test, usersIds_test_df, on='userId').drop(columns=['userId'])
ratings_test.columns = ['rating','productId', 'userId']


nb_users_train = ratings_train.userId.max()+1
print('nb users train', nb_users_train)
nb_movies_train = ratings_train.productId.max()+1
print('nb movies train', nb_movies_train)

nb_users_test = ratings_test.userId.max()+1
print('nb users test', nb_users_test)
nb_movies_test = ratings_test.productId.max()+1
print('nb movies test', nb_movies_test)


data_train = coo_array((ratings_train['rating'], (ratings_train['userId'], ratings_train['productId'])), shape=(nb_users_train, nb_movies_train)).toarray()
data_test = coo_array((ratings_test['rating'], (ratings_test['userId'], ratings_test['productId'])), shape=(nb_users_test, nb_movies_test)).toarray()



nb users train 80000
nb movies train 1400
nb users test 20000
nb movies test 1400


In [21]:
print('userId train size: ', len(usersIds_train))
print('userId test size: ', len(usersIds_test))
print('ratings train size: ', len(ratings_train))
print('ratings test size: ', len(ratings_test))

userId train size:  80000
userId test size:  20000
ratings train size:  12361871
ratings test size:  3106871


In [22]:
from collections import namedtuple
Predictions = namedtuple('Predictions',
    [
        'rbmf',
        'random',
        'k_medoid',
        'most_rated'
    ]
)


In [23]:
# RBMF

# dobiti representatives
representatives_rbmf = get_rbmf_representatives(k=k, data=data_train)

# dobiti X
_, x_rbmf = get_C_X(representatives=representatives_rbmf, data=data_train, ratings=ratings_train)

# dobiti predvidjanje ocena
# data_test_predicted = data_test[:, representatives] @ X <- ovo su rejtinzi
data_test_predicted_rbmf = data_test[:, representatives_rbmf] @ x_rbmf

In [24]:
# Random Sampling

# dobiti representatives
representatives_random = get_random_representatives(k=k, ratings=ratings_train)

# dobiti X
_, x_random = get_C_X(representatives=representatives_random, data=data_train, ratings=ratings_train)

# dobiti predvidjanje ocena
data_test_predicted_random = data_test[:, representatives_random] @ x_random

In [25]:
# Most Rated

# dobiti representatives
representatives_most_rating = get_most_rated_representatives(k=k,ratings=ratings_train)

# dobiti X
_, x_most_rating = get_C_X(representatives=representatives_most_rating, data=data_train, ratings=ratings_train)

# dobiti predvidjanje ocena
data_test_predicted_most_rating = data_test[:, representatives_most_rating] @ x_most_rating

In [26]:
# K - medoid

# dobiti representatives
representatives_cluster = get_cluster_representatives(k=k)

# dobiti X
_, x_cluster = get_C_X(representatives=representatives_cluster, data=data_train, ratings=ratings_train)

# dobiti predvidjanje ocena
data_test_predicted_cluster = data_test[:, representatives_cluster] @ x_cluster

In [27]:
data_predicted = Predictions(rbmf=data_test_predicted_rbmf, random=data_test_predicted_random, k_medoid=data_test_predicted_cluster, most_rated=data_test_predicted_most_rating)

### Prec @ 10

- uzeti najvecih 10 rejtinga koje smo predivideli i videti koliko tu ima zaista dobrih predloga (tacnije rejting >= 4)

In [28]:
# y_predicted odozgo za ovo koristiti 
# imamo matricu (userId, productId) rejtinga za svakog userId sortirati taj red po rejtingu i naci top 10 productId
def prec_k(data: np.ndarray, data_predicted: np.ndarray, k: int, treshold: float) -> float:
    nb_users, nb_products = data.shape
    
    top_k_indices = data_predicted.argsort(axis=1)[:, -k:]
    real_ratings = np.take_along_axis(data, top_k_indices, axis=1)
    nb_good_predictions = (real_ratings >= treshold).sum()
    return nb_good_predictions / (nb_users * k)




In [29]:
top_k = 10
print(f"prec@{top_k} rbmf: {prec_k(data=data_test, data_predicted=data_predicted.rbmf, k=top_k, treshold=4):.2f}")
print(f"prec@{top_k} random: {prec_k(data=data_test, data_predicted=data_predicted.random, k=top_k, treshold=4):.2f}")
print(f"prec@{top_k} k_medoid: {prec_k(data=data_test, data_predicted=data_predicted.k_medoid, k=top_k, treshold=4):.2f}")
print(f"prec@{top_k} most_rated: {prec_k(data=data_test, data_predicted=data_predicted.most_rated, k=top_k, treshold=4):.2f}")

prec@10 rbmf: 0.68
prec@10 random: 0.37
prec@10 k_medoid: 0.60
prec@10 most_rated: 0.74


### MAP (Mean Averege Precision)

- podeliti dataset na trening i test (u radu koriscen 80-20 split)
- na treningu naci C i X
- na testu izdvojiti reprezentativne filmove i predvideti ocene ostalih
- sortirati filmove po predvidjenoj oceni
- izracunati prosek Prec@i za svako i na kome je zaista dobar film

In [30]:
# y_predicted odozgo za ovo koristiti 
def mean_ap(data: np.ndarray, data_predicted: np.ndarray, treshold: float) -> float:
    nb_users, nb_products = data.shape
    sorted_indices = (-data_predicted).argsort(axis=1, )
    data_sorted = np.take_along_axis(data, sorted_indices, axis=1)
    good_movies = data_sorted >= treshold
    nb_good_prediction = np.cumsum(good_movies, axis=1)

    nb_movies = np.arange(1, nb_products+1)
    nb_movies = np.vstack([nb_movies]*nb_users)

    precisions = nb_good_prediction / nb_movies

    top_part = precisions * good_movies

    AP = top_part.sum(axis=1) / good_movies.sum(axis=1)

    return np.nansum(AP) / np.count_nonzero(~np.isnan(AP))

In [31]:
print(f"MAP rbmf: {mean_ap(data=data_test, data_predicted=data_predicted.rbmf, treshold=4):.2f}")
print(f"MAP random: {mean_ap(data=data_test, data_predicted=data_predicted.random, treshold=4):.2f}")
print(f"MAP k_medoid: {mean_ap(data=data_test, data_predicted=data_predicted.k_medoid, treshold=4):.2f}")
print(f"MAP most_rated: {mean_ap(data=data_test, data_predicted=data_predicted.most_rated, treshold=4):.2f}")

MAP rbmf: 0.39
MAP random: 0.23
MAP k_medoid: 0.34
MAP most_rated: 0.38


### AUC skor 
- podeliti dataset na trening i test (u radu koriscen 80-20 split)
- na treningu naci C i X
- na testu izdvojiti reprezentativne filmove i predvideti ocene ostalih
- predvidjanja podeliti na dobre i lose filmove
- iskoristiti scikit-learn funkciju za racunanje ove metrike

In [32]:
# y_predicted odozgo za ovo koristiti 
def auc_score(data: np.ndarray, data_predicted: np.ndarray, treshold: float) -> float:
    sorted_indices = data_predicted.argsort(axis=1, )
    data_sorted = np.take_along_axis(data, sorted_indices, axis=1)
    good_movies = data_sorted >= treshold
    number_of_good_before = np.cumsum(good_movies, axis=1)
    number_of_missordered_pairs = (number_of_good_before * ~good_movies).sum(axis=1)
    product_of_numbers = good_movies.sum(axis=1) * (~good_movies).sum(axis=1)
    auc = 1 - number_of_missordered_pairs / product_of_numbers

    return    np.nansum(auc) / np.count_nonzero(~np.isnan(auc))   


In [33]:
print(f"auc rbmf: {auc_score(data=data_test, data_predicted=data_predicted.rbmf, treshold=4):.2f}")
print(f"auc random: {auc_score(data=data_test, data_predicted=data_predicted.random, treshold=4):.2f}")
print(f"auc k_medoid: {auc_score(data=data_test, data_predicted=data_predicted.k_medoid, treshold=4):.2f}")
print(f"auc most_rated: {auc_score(data=data_test, data_predicted=data_predicted.most_rated, treshold=4):.2f}")

auc rbmf: 0.85
auc random: 0.74
auc k_medoid: 0.82
auc most_rated: 0.83
